## References

- https://einops.rocks
- [einsum is all you need: aladdin persson video](https://www.youtube.com/watch?v=pkVwUVEHmfI)
- [einsum is all you need: tim rocktaschel blog](https://rockt.ai/2018/04/30/einsum)

## Einsum notation

Einsum notation for each element in the output matrix, $C$ of shape [i, k], found from matrix multiplication of two matrices, $A$ of shape [i, j] and $B$ of shape [j, k], is defined as:

$$
C_{ik} = \sum_{j} A_{i,j} B_{j,k}
$$

In code (einops), this can be written as:

```python
c = einops.einsum(a, b, "i j, j k -> i k")
```

where the matrices to be operated on are the first arguments (`a, b`), 

and the einsum string (`"i j, j k -> i k"`) is the last argument. 

The einsum string contains space-separated values (`"i j"` for `a`),

where each value corresponds to a dimension (`"i"` for `a`'s rows and `"j"` for `a`'s columns) in the input matrices. 

The matrix arguments are comma-separated, and an arrow, `->` yields the space-separated dimensions of the output matrix (`"i k"`). Any dimension not specified in the output matrix (`"j"`) is summed over.

Of course, this dimension (`"j"`) must be repeated and of equal size in the input matrices: we can't perform the summation required in matrix multiplication on unequal dimensions.

In [1]:
"""Imports."""

import einops
import numpy as np
import torch

In [9]:
# Following the example above, let i=2, j=3, k=2

a = torch.tensor([[1, 2, 3], [4, 5, 6]])
b = torch.tensor([[1, 2], [3, 4], [5, 6]])
c = einops.einsum(a, b, "i j, j k -> i k")
print(c)

tensor([[22, 28],
        [49, 64]])
